## Convert Neuron X distributed checkpoint to Huggingace format for Inferencing

The output from the training job is saved as NeuronX checkpoint. In this notebook we will convert the neuronX distributed checkpoint into a .pt weights file which can be used for inferencing.

To begin with we will retrieve the path for the checkpoint from the model output and also path to llama7b config file, this can be retreived from Notebook 3. Prepare Dataset.ipynb job output.

In [ ]:
nxd_checkpoint_path = "s3://sagemaker-us-east-2-365792799466/neuronx_llama_experiment/checkpts/step10/model/" # Checkpoint is saved as part of Notebook 2

In [ ]:
import sagemaker 

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [ ]:
hyperparameters = {}

hyperparameters["n_layers"] = 80
hyperparameters["pp_size"] = 8
hyperparameters["tp_size"] = 8
hyperparameters["input_dir"] = "/opt/ml/input/data/checkpoint"
hyperparameters["convert_to_full_model"] = ""
hyperparameters["output_dir"] = "/opt/ml/model"
hyperparameters["access_token"] = "hf_xxxx"

In [ ]:
docker_image = "763104351884.dkr.ecr.us-east-2.amazonaws.com/pytorch-training-neuronx:1.13.1-neuronx-py310-sdk2.17.0-ubuntu20.04"

In [ ]:
from sagemaker.pytorch import PyTorch
# Need to check if this works on multinode with torchrun.
estimator = PyTorch(
    base_job_name="neuronx-convert-checkpoint-to-hf",
    source_dir="./scripts",
    entry_point="convert_checkpoints.py",
    role=role,
    image_uri=docker_image,
    instance_count=1,
    instance_type="ml.trn1.32xlarge",
    sagemaker_session=sess,
    volume_size=1024,
    hyperparameters=hyperparameters,
    debugger_hook_config=False,
    disable_output_compression=True,
    keep_alive_period_in_seconds=600,
)

In [ ]:
estimator.fit({"checkpoint":nxd_checkpoint_path})

In [ ]:
model_path = estimator.model_data['S3DataSource']['S3Uri']

In [ ]:
print(f"You can find the converted weights here {model_path}")